In [16]:
from tinygrad.tensor import Tensor
import tinygrad.nn as nn
from tinygrad.nn.datasets import cifar
from tinygrad.nn.optim import Adam
from tinygrad.nn.state import get_parameters
from tinygrad.dtype import dtypes
import numpy as np
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader

In [17]:
class BasicBlock():
    outchannel_ratio = 1
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        # https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
        # eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        # https://docs.tinygrad.org/nn/#tinygrad.nn.BatchNorm
        # eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        self.bn1 = nn.BatchNorm(in_channels)
        # https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
        # dilation=1, groups=1
        # https://docs.tinygrad.org/nn/#tinygrad.nn.Conv2d
        # dilation=1, groups=1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm(out_channels)
        self.downsample = downsample
        self.stride = stride

    def __call__(self, x):
        out = self.bn1(x)
        #print(f"bn1 shape = {out.size()}")
        out = self.conv1(out)
        #print(f"conv1 shape = {out.size()}")
        out = self.bn2(out)
        #print(f"bn2 shape = {out.size()}")
        out = out.relu()
        #print(f"relu shape = {out.size()}")
        out = self.conv2(out)
        #print(f"conv2 shape = {out.size()}")
        
        out = self.bn3(out)
        #print(f"bn3 shape = {out.size()}")

        if self.downsample is not None:
            shortcut = self.downsample(x)
            featuremap_size = shortcut.size()[2:4]
        else:
            shortcut = x
            featuremap_size = out.size()[2:4]

        batch_size = out.size()[0]
        residual_channel = out.size()[1]
        shortcut_channel = shortcut.size()[1]

        if residual_channel != shortcut_channel:
            padding = Tensor.zeros(batch_size, residual_channel - shortcut_channel, featuremap_size[0], featuremap_size[1])
            out = out + Tensor.cat(shortcut, padding, dim=1)
        else:
            out = out + shortcut
        #print(f"layer out shape = {out.size()}", end="\n\n")
        return out


class Bottleneck():
    outchannel_ratio = 4
    def __init__(self, in_planes, planes, stride=1, downsample=None):
        self.bn1 = nn.BatchNorm(in_planes)
        # https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
        # stride=1, padding=0
        # https://docs.tinygrad.org/nn/#tinygrad.nn.BatchNorm
        # stride=1, padding=0
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn3 = nn.BatchNorm(planes)
        self.conv3 = nn.Conv2d(planes, planes * Bottleneck.outchannel_ratio, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm(planes * Bottleneck.outchannel_ratio)
        self.relu = Tensor.relu
        self.downsample = downsample
        self.stride = stride

    def __call__(self, x):
        out = self.bn1(x)
        out = self.conv1(out)
        
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)

        out = self.bn3(out)
        out = self.relu(out)
        out = self.conv3(out)
        
        out = self.bn4(out)

        if self.downsample is not None:
            shortcut = self.downsample(x)
            featuremap_size = shortcut.size()[2:4]
        else:
            shortcut = x
            featuremap_size = out.size()[2:4]

        batch_size = out.size()[0]
        residual_channel = out.size()[1]
        shortcut_channel = shortcut.size()[1]

        if residual_channel != shortcut_channel:
            padding = Tensor.zeros(batch_size, residual_channel - shortcut_channel, featuremap_size[0], featuremap_size[1])
            
            out += Tensor.cat(shortcut, padding, 1)
        else:
            out += shortcut

        return out


class PyramidNet:
    def __init__(self, num_classes, depth, alpha, bottleneck=False):
        if depth not in [18, 34, 50, 101, 152, 200]:
            if bottleneck:
                block = Bottleneck
                temp_cfg = (depth - 2) // 12
            else:
                block = BasicBlock
                temp_cfg = (depth - 2) // 8
            layers = [temp_cfg, temp_cfg, temp_cfg, temp_cfg]
            print('=> the layer configuration for each stage is set to', layers[depth])
        else:
            block = BasicBlock if depth <= 34 and not bottleneck else Bottleneck
            if depth == 18:
                layers = [2, 2, 2, 2]
            elif depth in [34, 50]:
                layers = [3, 4, 6, 3]
            elif depth == 101:
                layers = [3, 4, 23, 3]
            elif depth == 152:
                layers = [3, 8, 36, 3]
            else:
                layers = [3, 24, 36, 3]

        self.in_planes = 64            
        self.addrate = alpha / sum(layers)

        self.input_featuremap_dim = self.in_planes
        self.conv1 = nn.Conv2d(3, self.in_planes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_planes)

        self.featuremap_dim = self.input_featuremap_dim 
        self.layer1 = self.pyramidal_make_layer(block, layers[0])
        self.layer2 = self.pyramidal_make_layer(block, layers[1], stride=2)
        self.layer3 = self.pyramidal_make_layer(block, layers[2], stride=2)
        self.layer4 = self.pyramidal_make_layer(block, layers[3], stride=2)

        self.final_featuremap_dim = self.input_featuremap_dim
        self.bn_final = nn.BatchNorm2d(self.final_featuremap_dim)
        self.avgpool = lambda x: x.avg_pool2d(7)
        self.fc = nn.Linear(self.final_featuremap_dim, num_classes)

    def pyramidal_make_layer(self, block, block_depth, stride=1):
        downsample = None
        if stride != 1:
            downsample = lambda x: x.avg_pool2d((2, 2), stride=(2, 2), ceil_mode=True)

        layers = []
        self.featuremap_dim += self.addrate
        layers.append(block(self.input_featuremap_dim, int(round(self.featuremap_dim)), stride, downsample))
        for i in range(1, block_depth):
            temp_featuremap_dim = self.featuremap_dim + self.addrate
            layers.append(block(int(round(self.featuremap_dim)) * block.outchannel_ratio, int(round(temp_featuremap_dim)), 1))
            self.featuremap_dim  = temp_featuremap_dim
        self.input_featuremap_dim = int(round(self.featuremap_dim)) * block.outchannel_ratio
        return layers

    def __call__(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = x.relu()
        x = x.max_pool2d(kernel_size=3, stride=2, padding=1)

        for layer in self.layer1:
            x = layer(x)
        for layer in self.layer2:
            x = layer(x)
        for layer in self.layer3:
            x = layer(x)
        for layer in self.layer4:
            x = layer(x)

        x = self.bn_final(x)
        #print(x.size())
        x = x.relu()
        #print(x.size())
        x = x.avg_pool2d(7)
        #print(x.size())
        x = x.view(x.size(0), -1)
        #print(x.size())
        #print(x)
        x = self.fc(x)
        #print(x.size())
        return x
    
        


In [25]:
# def resize_data(data, target_size=224):
#     new_data = None
#     for image_t in tqdm(data):
#         image = Image.fromarray(np.transpose(image_t.numpy(), (1, 2, 0))).resize((target_size, target_size))
#         image_np = np.transpose(np.array(image), (2, 0, 1))
#         if new_data is None:
#             new_data = Tensor(image_np, dtype=dtypes.uint8)
#         else:
#             new_data.stack(Tensor(image_np, dtype=dtypes.uint8))
#     return new_data

# X_train, Y_train, X_test, Y_test = cifar()
# X_train = resize_data(X_train)
# X_test = resize_data(X_test)
X_train, Y_train, X_test, Y_test = cifar()
print(X_train.size())

(50000, 3, 32, 32)


In [ ]:
num_classes = 200
depth = 18
alpha = 48
batch_size = 32
epochs = 1000
learning_rate = 0.001

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    lambda x: np.array(x)
])

train_dataset = torchvision.datasets.ImageFolder(root='C:/Users/elect/Documents/SEM9/Advanced Machine Learning/tiny-imagenet-200/train', transform=transform)
val_dataset = torchvision.datasets.ImageFolder(root='C:/Users/elect/Documents/SEM9/Advanced Machine Learning/tiny-imagenet-200/val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = PyramidNet(num_classes=num_classes, depth=depth, alpha=alpha)

optimizer = Adam(get_parameters(model), lr=learning_rate)

with Tensor.train():
    for i, (images, labels) in tqdm(enumerate(train_loader)):
        images = Tensor(images.numpy().transpose((0, 3, 1, 2)))
        labels = Tensor(labels.numpy())
        out = model(images)
        loss = out.sparse_categorical_crossentropy(labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 100 == 99:
            pred = out.argmax(axis=-1)
            acc = (pred == labels).mean()
            print(f"Loss: {loss.numpy()} | Accuracy: {acc.numpy()}")



100it [02:40,  1.64s/it]

Loss: 5.153182029724121 | Accuracy: 0.0


200it [05:16,  1.86s/it]

Loss: 5.136722087860107 | Accuracy: 0.03125


300it [07:53,  1.60s/it]

...

600it [15:49,  2.11s/it]

Loss: 4.73364782333374 | Accuracy: 0.0625


700it [18:33,  1.69s/it]

Loss: 4.834665775299072 | Accuracy: 0.0625


760it [20:21,  1.94s/it]